In [1]:
import pandas as pd
import numpy as np
import math
from nptdms import TdmsFile
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# File paths
file_path = "/Users/shubhangchauhan/Desktop/Procyon Data/TaehwaV3/11_06_2025/10nmSweep1_11_06_2020_12_13 0.tdms"
csv_filtered_path = "/Users/shubhangchauhan/Desktop/single_file_filtered.csv"
html_output_path = "/Users/shubhangchauhan/Desktop/single_file_filtered_grid.html"

tdms_file = TdmsFile.read(file_path)
df = tdms_file.as_dataframe()
df.columns = df.columns.str.replace(" ", "_")
df.columns = df.columns.str.replace("/'Calculated_Data'/", "", regex=True)
df.columns = df.columns.str.replace("'", "", regex=False)
df["Index"] = range(len(df))

torque_centers = np.arange(0, 50, 5)
Vdc = 51
threshold_voltage = 0.9 * Vdc * 0.666 

# Functions
def assign_torque_group(torque):
    if pd.isna(torque):
        return None
    for center in torque_centers:
        if center - 0.2 <= torque <= center + 0.2:
            return center
    return None

# Read and clean
tdms_file = TdmsFile.read(file_path)
df = tdms_file.as_dataframe()
df.columns = df.columns.str.replace(" ", "_")
df.columns = df.columns.str.replace(f"/'Calculated_Data'/", "", regex=True)
df.columns = df.columns.str.replace("'", "", regex=False)
df.EFFI1 = df.EFFI1 * 100.0

# Filtering
required_cols = ["CAN_Id_Ref", "CAN_Iq_Ref", "Torque", "CAN_Vstator_V", "CAN_Iq_in", "CAN_Id_in", "CAN_Ld_uH", "CAN_Lq_uH"]
if not all(col in df.columns for col in required_cols):
    raise ValueError("Missing required columns.")

df_filtered = df[(df["CAN_Id_Ref"] != 0) & (df["CAN_Iq_Ref"] != 0)]
df_filtered = df_filtered[~((df_filtered["CAN_Iq_Ref"] < 0) | (df_filtered["CAN_Id_Ref"] > 0) |
                            (df_filtered["CAN_Iq_in"] < 0) | (df_filtered["CAN_Id_in"] > 0))]

df_filtered["Torque"] = pd.to_numeric(df_filtered["Torque"], errors='coerce').abs()
df_filtered = df_filtered[~df_filtered["Torque"].isna()]

if "Speed" in df_filtered.columns:
    df_filtered["Speed"] = pd.to_numeric(df_filtered["Speed"], errors='coerce').abs()
    df_filtered = df_filtered[df_filtered["Speed"] <= 3000]
    df_filtered["Speed_Diff"] = df_filtered["Speed"].diff()
    df_filtered = df_filtered[(df_filtered["Speed_Diff"] == 0) | (df_filtered["Speed_Diff"].isna())]
    df_filtered.drop(columns=["Speed_Diff"], inplace=True)

df_filtered["CAN_Ld_uH"] = pd.to_numeric(df_filtered["CAN_Ld_uH"], errors="coerce")
df_filtered["CAN_Lq_uH"] = pd.to_numeric(df_filtered["CAN_Lq_uH"], errors="coerce")
df_filtered = df_filtered.dropna(subset=["CAN_Ld_uH", "CAN_Lq_uH"])
df_filtered = df_filtered[(df_filtered["CAN_Ld_uH"] == 46.3) & (df_filtered["CAN_Lq_uH"] == 73.48)]

for col in ["CAN_Iq_in", "CAN_Iq_Ref", "CAN_Id_in", "CAN_Id_Ref"]:
    df_filtered[col] = pd.to_numeric(df_filtered[col], errors='coerce')

df_filtered = df_filtered.dropna(subset=["CAN_Iq_in", "CAN_Iq_Ref", "CAN_Id_in", "CAN_Id_Ref"])
df_filtered = df_filtered[
    ((df_filtered["CAN_Iq_in"] - df_filtered["CAN_Iq_Ref"]).abs() <= 10) &
    ((df_filtered["CAN_Id_in"] - df_filtered["CAN_Id_Ref"]).abs() <= 10)
]

if "CAN_Bandwidth" in df_filtered.columns:
    df_filtered = df_filtered[df_filtered["CAN_Bandwidth"] == 500]

df_filtered = df_filtered[df_filtered["CAN_Vstator_V"].abs() < threshold_voltage]
df_filtered["Torque_Group"] = df_filtered["Torque"].apply(assign_torque_group)
df_filtered.insert(0, "Index", range(1, len(df_filtered) + 1))

# Compute power and efficiency before saving
df_filtered["Torque"] = pd.to_numeric(df_filtered["Torque"], errors='coerce')
df_filtered["Speed"] = pd.to_numeric(df_filtered.get("Speed", np.nan), errors='coerce')
df_filtered["Udc4"] = pd.to_numeric(df_filtered.get("Udc4", np.nan), errors='coerce')
df_filtered["Idc4"] = pd.to_numeric(df_filtered.get("Idc4", np.nan), errors='coerce')

df_filtered["Pout"] = df_filtered["Torque"] * df_filtered["Speed"] * 2 * np.pi / 60  # W
df_filtered["Pin"] = df_filtered["Udc4"] * df_filtered["Idc4"]  # W
df_filtered["Efficiency"] = np.where(df_filtered["Pin"] != 0, df_filtered["Pout"] / df_filtered["Pin"], np.nan)

# Remove entries where efficiency > 1
df_filtered = df_filtered[df_filtered["Efficiency"] <= 1]

# --- Optional DBSCAN Filtering ---
USE_DBSCAN_FILTER = False  # Toggle this to enable/disable DBSCAN

if USE_DBSCAN_FILTER:
    from sklearn.cluster import DBSCAN
    from sklearn.preprocessing import StandardScaler
    from sklearn.neighbors import NearestNeighbors

    filtered_groups = []

    for tg, group in df_filtered.groupby("Torque_Group"):
        if len(group) < 10:
            continue

        features = group[["CAN_Id_in", "CAN_Iq_in"]].dropna()
        if features.empty:
            continue

        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features)

        neigh = NearestNeighbors(n_neighbors=5)
        nbrs = neigh.fit(features_scaled)
        distances, _ = nbrs.kneighbors(features_scaled)
        k_distances = distances[:, -1]

        eps_val = max(np.percentile(k_distances, 90), 0.1)

        db = DBSCAN(eps=eps_val, min_samples=5).fit(features_scaled)
        labels = db.labels_

        mask = labels != -1
        filtered = group.loc[features.index[mask]].copy()
        filtered_groups.append(filtered)

    df_filtered = pd.concat(filtered_groups, ignore_index=True)
    print("DBSCAN filtering applied to single-file data.")
else:
    print("DBSCAN filtering skipped.")


# Save filtered
df_filtered.to_csv(csv_filtered_path, index=False)
print(f"Filtered file saved to: {csv_filtered_path}")


DBSCAN filtering applied to single-file data.
Filtered file saved to: /Users/shubhangchauhan/Desktop/single_file_filtered.csv


In [ ]:
df = df_filtered.copy()

# List of individual variables to plot
single_var_plots = ["CAN_Id_in", "CAN_Iq_in", "CAN_Vstator_V", "CAN_Vd_Out", "CAN_Vq_Out", "CAN_Istator_A", "CAN_Id_Ref","CAN_Iq_Ref"]

# List of pairs to overlay in one subplot: (var1, var2)
dual_var_plots = [
    ("Torque", "Speed") #,
    #("CAN_Istator_A", "EFFI1")
]

# Plotting
total_plots = len(single_var_plots) + len(dual_var_plots)
cols_per_row = 2
rows = math.ceil(total_plots / cols_per_row)

titles = [col for col in single_var_plots] + [f"{a} + {b}" for a, b in dual_var_plots]
fig = make_subplots(
    rows=rows,
    cols=cols_per_row,
    subplot_titles=titles,
    specs=[[{"secondary_y": True} for _ in range(cols_per_row)] for _ in range(rows)]
)

plot_idx = 0

# Plot single variable plots
for col in single_var_plots:
    if col in df.columns:    
        row, col_idx = divmod(plot_idx, cols_per_row)
        fig.add_trace(go.Scatter(x=df["Index"], y=pd.to_numeric(df[col], errors="coerce"),
                                 name=col, mode="lines"),
                      row=row+1, col=col_idx+1)
        plot_idx += 1

# Plot dual variable overlays
for var1, var2 in dual_var_plots:
    row, col_idx = divmod(plot_idx, cols_per_row)
    row += 1
    col_idx += 1
    yaxis_base = f"y{(row - 1) * cols_per_row + col_idx}"
    yaxis_secondary = f"y{(row - 1) * cols_per_row + col_idx}2"

    # Primary Y-axis trace
    if var1 in df.columns:
        fig.add_trace(go.Scatter(
            x=df["Index"],
            y=pd.to_numeric(df[var1], errors="coerce"),
            name=var1,
            mode="lines",
            yaxis=yaxis_base
        ), row=row, col=col_idx,secondary_y=False)

    # Secondary Y-axis trace
    if var2 in df.columns:
        fig.add_trace(go.Scatter(
            x=df["Index"],
            y=pd.to_numeric(df[var2], errors="coerce"),
            name=var2,
            mode="lines",
            yaxis=yaxis_secondary
        ), row=row, col=col_idx,secondary_y=True)

    fig.update_yaxes(title_text=var1, row=row, col=col_idx)
    fig.update_yaxes(title_text=var2, row=row, col=col_idx, secondary_y=True)
    plot_idx += 1


fig.update_layout(
    height=200 * rows,
    width=720 * cols_per_row,
    title=f"Custom Data Subplots: {file_path.split('/')[-1]}",
    showlegend=False
)

# Save outputs
df_out_cols = ["Index"] + list(set(single_var_plots + [x for pair in dual_var_plots for x in pair]))
df[df_out_cols].to_csv(csv_filtered_path, index=False)
fig.write_html(html_output_path)

print("Saved CSV and HTML plot.")

Saved CSV and HTML plot.
